# Computational Genetics: Statistics, Supervised and Unsupervised Learning
### Final Report
Olyssa Sperling (614143)
olyssa.sperling@student.hu-berlin.de
Module: Systems Biology: Computational Analysis and Interpretation of High-throughput Data

#### Background
##### 1) CpG methylation data:
CpG methylation data refers to information about the methylation status of cytosine-guanine dinucleotides (CpG sites) in DNA. DNA methylation is an epigenetic modification where a methyl group is added to the 5th carbon of the cytosine ring, primarily at CpG sites. Methylation patterns are crucial for normal development and cellular differentiation. CpG methylation data is often presented as a table with rows as CpG sites and columns as samples.

#####
##### 2) CpG methylation influenced by age:
With age, CpG methylation undergoes a combination of global hypomethylation, localized hypermethylation, and epigenetic drift. These patterns are not just markers of aging but also play active roles in age-related diseases. By studying these changes, researchers can better understand the molecular basis of aging and develop interventions to promote healthy aging.

Key aspects are:
* **Global Hypomethylation**: As individuals age, there is a general loss of methylation across the genome, particularly in repetitive elements and intergenic regions. Hypomethylation in these regions can lead to genomic instability, activation of transposable elements, and inappropriate gene expression


* **Local Hypermethylation**: While global hypomethylation occurs, some specific CpG sites, especially in promoter regions of certain genes, become hypermethylated with age. Hypermethylation often occurs in tumor suppressor genes and pathways involved in cell cycle regulation, contributing to age-related diseases, including cancer.

* **Epigenetic drift**: Epigenetic drift refers to the stochastic (random) changes in DNA methylation patterns that accumulate with age, leading to increased inter-individual variability in methylation profiles over time. This drift is influenced by both genetic and environmental factors, such as diet, smoking, and exposure to pollutants.

* **Age-Associated Differentially Methylated Positions (aDMPs)**: Certain CpG sites show predictable changes in methylation with age, known as age-associated differentially methylated positions (aDMPs). These aDMPs are often enriched in regulatory regions and genes involved in development, such as those in the HOX gene family.

*  **Impact on Health**: 
    * *Cancer*: Age-related methylation changes can lead to hypermethylation of tumor suppressor genes and hypomethylation of oncogenes.
    * *Neurodegeneration*: Methylation changes in genes involved in neuronal function and inflammation contribute to conditions like Alzheimer's disease.
    * *Immune System Decline*: Age-related methylation patterns in immune-related genes contribute to immunosenescence (decline of immune function).